<a href="https://colab.research.google.com/github/harshitadd/CrossSiloFLDemo/blob/master/(Larger_minibatch)_PreVID_PySyft_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install syft --quiet

In [2]:
##Importing Mini Batch Data 
import os 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!pip install pydicom --quiet
import pydicom 
import cv2
import matplotlib.pyplot as plt 

In [4]:
dcm_path=os.listdir('/content/drive/My Drive/Fed_Covid/minibatch_large/')
dcm_data={}

alpha = 1.5 # Contrast control (1.0-3.0)
beta = 0 # Brightness control (0-100)

ctr=0
for file in dcm_path:
  name = '/content/drive/My Drive/Fed_Covid/minibatch_large/' + file
  temp = pydicom.dcmread(name)
  image = temp.pixel_array
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
  adjusted = cv2.resize(image,(64,64))
  dcm_data[file]=adjusted
  ctr+=1

In [5]:
import csv 
labels=[]
with open('/content/drive/My Drive/Fed_Covid/stage_2_train_labels.csv','r') as file:
  reader = csv.reader(file)
  for row in reader:
    labels.append(row)

In [6]:
from sklearn.preprocessing import MinMaxScaler 
import numpy as np
scaler = MinMaxScaler()
pid=[]
dicom=[]
label=[]
cid = 0
for PID in labels:
  for key in dcm_data:
    if(key[:-4]==PID[0]):
      l=[]
      for val in dcm_data[key]:
        l.append(scaler.fit_transform(val))
      l = np.reshape(l,(3,64,64))
      dicom.append(l)
      label.append(int(PID[5]))
      

In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import syft as sy
from torch.utils.data import TensorDataset, DataLoader
import time
import copy
import numpy as np
from torch.utils.data import Dataset
from syft.frameworks.torch.fl import utils
from syft.workers.websocket_client import WebsocketClientWorker

In [8]:
class Arguments():
    def __init__(self):
        self.batch_size = 4
        self.test_batch_size = 4
        self.epochs = 5
        self.lr = 0.01
        self.momentum = 0.5
        self.seed = 1
        self.log_interval = 10
        self.save_model = False

args = Arguments()

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(dicom,label,test_size=0.3)

In [10]:
import pandas as pd 
test_df = pd.DataFrame()
test_df['features']=x_test
test_df['labels']=y_test

In [11]:
x_train= np.array(x_train)
y_train= np.array(y_train)
x_test= np.array(x_test)
y_test= np.array(y_test)

In [12]:
class data_maker(Dataset):
  def __init__(self, images, labels):
        self.data = images 
        self.targets = labels 

        self.to_torchtensor()
        
  def __len__(self):
      return len(self.data)

  def to_torchtensor(self):      
      self.data=torch.from_numpy(self.data)
      self.labels=torch.from_numpy(self.targets)
  
  def __getitem__(self, idx):
      sample=self.data[idx]
      target=self.targets[idx]
      return sample,target

In [13]:
hook = sy.TorchHook(torch)
hospital = sy.VirtualWorker(hook, id="hospital")  
clinic = sy.VirtualWorker(hook, id="clinic")  

In [14]:
federated_train_data = data_maker(x_train,y_train).federate((hospital,clinic))

In [15]:
federated_train_loader = sy.FederatedDataLoader(federated_train_data,batch_size=args.batch_size)
test_data = data_maker(x_test,y_test)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.test_batch_size)

In [16]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        # Input - (32,1024,1024)
  
        self.conv1 = nn.Conv2d(3,32, kernel_size=8, stride = 2)  #I/P - 1*32
        self.pool1 = nn.MaxPool2d(kernel_size=4,stride=2,padding=0)  
        self.conv2 = nn.Conv2d(32, 64, 8)
        self.pool2 = nn.MaxPool2d(4,2,padding=0)
        self.fc1 = nn.Linear(256, 64*64)
        self.fc2 = nn.Linear(64*64, 64)
        self.fc3 = nn.Linear(64, 2)

    def forward(self, x):
        
        x = F.relu(self.conv1(x))
        # Input of 1*1024*1024 going to 32*1024*1024  - 1*64*64 going to 32*64*64
        x = self.pool1(x)
        # Input of 32*1024*1024 going to 32*256*256 - 32*64*64 going to 32*16*16
        x = F.relu(self.conv2(x))
        # Input of 32*256*256 to 64*256*256 - 32*16*16 to 64*16*16
        x = self.pool2(x)
        # Input of 64*256*256 to 64*64*64 - 64*16*16 going to 64*4*4

        x = x.view(-1,256)
     
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return F.log_softmax(x,dim=1)

In [17]:
def train(args, model,train_loader, optimizer, epoch):
    model.train()
    loss = nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        out = loss(output, target)
        out.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        
        if batch_idx % args.log_interval == 0:
            final_loss = out.get() # <-- NEW: get the loss back
            print('For Training Epoch: ' + str(epoch) + 'loss is :' + str(final_loss) + 'at ' + str(data.location))

In [18]:
def test(args, model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    loss = nn.CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            output = model(data)
            out = loss(output,target)
            test_loss +=out
            pred = output.argmax(1, keepdim=True) # get the index of the max log-probability 
            correct += pred.eq(target.view_as(pred))

    test_loss /= len(test_loader.dataset)

    print('For Test Loss:' + str(test_loss))

In [19]:
model = Net().double()
optimizer = optim.SGD(model.parameters(), lr=args.lr) 
for epoch in range(1, 10):
    model.train()
    loss = nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(federated_train_loader): # <-- now it is a distributed dataset
        model.send(data.location) # <-- NEW: send the model to the right location
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        out = loss(output, target)
        out.backward()
        optimizer.step()
        model.get() # <-- NEW: get the model back
        
        if batch_idx % args.log_interval == 0:
            final_loss = out.get() # <-- NEW: get the loss back
            print('Train Epoch:' + str(epoch) + 'loss is :' + str(final_loss))
  



Train Epoch:1loss is :tensor(0.6768, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.6071, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.5745, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.5746, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.3302, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.5712, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.5548, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.2796, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.1416, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.2194, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.2314, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.5543, dtype=torch.float64, requires_grad=True)
Train Epoch:1loss is :tensor(0.5498, dtype=torch.float64, requir

In [20]:
test(args, model, test_loader)

For Test Loss:tensor(0.1031, dtype=torch.float64)
